In [2]:
import numpy as np
import tensorflow as tf
import random
import dqn
from collections import deque
import gym
from gym import wrappers
env = gym.make('CartPole-v0')

In [3]:
input_size = env.observation_space.shape[0]
output_size = env.action_space.n
dis =0.9
REPLAY_MEMORY = 50000

In [4]:
class DQN:
    
    def _init_(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        self._build_network()
        
    def _build_network(self, h_size=10, l_rate=1e-1):
        with tf.variable_scope(self.net_name):
            self._X = tf.placeholder(
                tf.float32, [None, self.input_size], name="input_x") #None은 1개 또는 여러개 가능
            
            # 1st layer of weights
            W1 = tf.get_variable("W1", shape=[self.input_size, h_size], #input_size는 입력, h_size는 hidden layer로의 출력: hidden size
                                initializer=tf.contrib.layers.xavier_initializer())
            layer1 = tf.nn.tanh(tf.matmul(self._X, W1)) # X 와 W1 계산 후 activation function 으로 묶음(tanh)
            
            #2nd layer of weights
            W2 = tf.get_variable("W2",shape=[h_size, self.output_size], # W1 에서 나온 output을 입력으로, 원하는 output size로
                                initializer=tf.contrib.layers.xavier_initializer())
            
            #Q prediction
            self._Qpred = tf.matmul(layer1, W2)
        
        #policy
        self._Y = tf.placeholder(
            shape=[None, self.output_size], dtype=tf.float32)
        
        #loss function
        self._loss = tf.reduce_mean(tf.square(self._Y - self._Qpred)) #linear regression 에서 썼던 방법과 같음
        
        #learning
        self._train = tf.train.AdamOptimizer(
            learning_rate=l_rate).minimize(self._loss)
    
    def predict(self, state): 
        x = np.reshape(state, [1,self.input_size])
        return self.session.run(self._Qpred, feed_dict={self._X: x})
    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={
            self._X: x_stack, self._Y: y_stack
        })


In [5]:
def simmple_replay_train(DQN, train_batch):
    x_stack = np.empty(0).reshape(0, DQN.input_size)
    y_stack = np.empty(0).reshape(0, DQN.output_size)
    
    #get stored information from the buffer
    for state, action, reward, next_state, done in train_batch:
        Q = DQN.predict(state)
        
        #terminal?
        if done:
            Q[0, action] = reward
        else:
            #Obtain the Q' values by feeding the new state through our network
            Q[0, action] = reward + dis * np.max(DQN.predict(next_state))
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
        
    #train our network using target and predicted Q values on each episode
    return DQN.update(x_stack, y_stack)


In [6]:
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, reshape(0, input_size))
    y_stack = np.empty(0).reshape(0, reshape(0, output_size))
    
   #Get stored information from the buffer
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        
        if done:
            Q[0, action] = reward
        else:
            #get target from target DQN (Q')
            Q[0,action] = reward + dis * np.max(targetDQN.prredict(next_state))
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack({x_stack, state})
        
    #Train our network using target and predicted Q values on each episode
    return mainDQN.update(x_stack, y_stack)

In [7]:
def ddqn_reply_train(mainDQN, targetDQN, train_batch):
    '''
    Double DQN implementation
    :param mainDQN : main DQN
    :param targetDQN : target DQN
    :param train_batch : minibatch for train
    :return loss
    '''
    
    
    x_stack = np.empty(0).reshape(0, mainDQN.input_size)
    y_stack = np.empty(0).reshape(0, mainDQN.output_size)
    
    #Get stored information from the buffer
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        
        if done:
            Q[0, action] = reward
        else:
            #Double DQN : y= r + gamma * targetDQN(s')[a] where
            # a = argmax(mainDQN(s'))
            Q[0,action] = reward + dis * targetDQN.prredict(next_state)[
                0, np.argmax(mainDQN.predict(next_state))]
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack({x_stack, state})
        
    #Train our network using target and predicted Q values on each episode
    return mainDQN.update(x_stack, y_stack)

In [8]:
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    
    #copy variables src_scope to dest_scope
    op_holder=[]
    
    src_vars = tf.get_collection(
        tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    dest_vars = tf.get_collection(
        tf.GraphKeys.TRAINABLE_VARIABLES, scope=dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value()))
    
    return op_holder

In [9]:
def bot_play(mainDQN):
    #see our trained network in action
    s = env.reset()
    reward_sum = 0
    while True:
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print("Total score; {}".format(reward_sum))
            break

In [10]:
def main():
    max_episodes = 5000
    
    #store thr previous observations in replay memory
    replay_buffer = deque()
    
    with tf.Session() as sess:
        mainDQN = dqn.DQN(sess, input_size, output_size, name="main")
        targetDQN = dqn.DQN(sess, input_size, name="target")
        tf.global_variables_initializer().run()
        
        #initial copy q_net -> target_net
        copy_ops = get_copy_var_ops(dest_scope_name="target",
                                   src_scope_name="main")
        sess.run(copy_ops)
        
        for episode in range(max_episodes):
            e = 1./((episode/10)+1)
            done = False
            step_count = 0
            state = env.reset()
            
            while not done:
                if np.random.rand(1) < e:
                    action = env.action_space.sample()
                else:
                    # choose an action by greedy from the Q-network
                    action = np.argmax(mainiDQN.predict(state))
                
                # get new state and reward from environment
                next_state, reward, done, _ =env.step(action)
                if done: #penalty
                    reward = -100
                    
                # save the experience to our buffer
                replay_buffer.append((state, action, reward, next_state, done))
                if len(replay_buffer) > REPLAY_MEMORY:
                    replay_buffer.popleft() # 버퍼 커지면 앞 삭제
                    
                state = next_state
                step_count += 1
                if step_count > 10000: # good enough
                    break
                    
            print("Episode: {} steps: {}".format(episode, step_count))
            if step_count > 10000:
                pass
                break
            
            if episode % 10 ==1: #train every 10 episode
                # get random batch of experiences
                for _ in range(50):
                    minibatch = random.sample(reply_buffer, 10)
                    loss, _ = ddqn_reply_train(mainDQN, targetDQN, minibatch)
                
                print("Loss: ",loss)
                #copy q_net -> target_net
                sess.run(copy_ops)

In [11]:
if __name__ =="__main__":
    main()

NameError: name 'dqn' is not defined